In [1]:
import os
import time
import random
import shutil


random.seed(42)

def check_folder(folder_path):
    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
        time.sleep(0.2)
        os.makedirs(folder_path)
    else:
        os.makedirs(folder_path)

def path_correct(old_path=None):
    new_path = ''
    for i in old_path.split('\\'):
        new_path += i + '/'
    return new_path

output_path = 'D:/darknet/build/darknet/x64/fish'  # 英文專案名稱
data_dir = "C:/Users/user/Desktop/pr/img"          # 標籤資料
# train, test, validation
split_rate = [0.9, 0.1]
output_dir = ['train','test','validation']


cfg_obj_names = "obj.names"
cfg_obj_data = "obj.data"
cfg_train_txt = "train.txt"
cfg_test_txt = "test.txt"

# ===============================================output_path
if os.path.isdir(output_path):
    shutil.rmtree(output_path)

    
# ===============================================cfg weights train test
for i in range(len(split_rate)):
    check_folder(os.path.join(output_path, output_dir[i]))
check_folder(os.path.join(output_path, 'cfg'))
check_folder(os.path.join(output_path, 'weights'))


# ===============================================obj.names
with open(os.path.join(data_dir, 'classes.txt'), 'r') as f1:
    with open(os.path.join(output_path, 'cfg', cfg_obj_names), 'w') as f2:
        f2.write(f1.read())
    f2.close()
f1.close()

with open(os.path.join(output_path, 'cfg', cfg_obj_names), 'r') as f:
    classes = f.read()
f.close()

# ===============================================obj.data
with open(os.path.join(output_path, 'cfg', cfg_obj_data), 'w') as f:  
    f.write("classes = " + str(len(classes.split())) + "\n")
    f.write(r"train = " + path_correct(os.path.abspath(os.path.join(output_path, 'cfg', 'train.txt')))[:-1] + "\n")
    f.write(r"valid = " + path_correct(os.path.abspath(os.path.join(output_path, 'cfg', 'test.txt')))[:-1] + "\n")
    f.write(r"names = " + path_correct(os.path.abspath(os.path.join(output_path, 'cfg', 'obj.names')))[:-1] + "\n")
    f.write(r"backup = " + path_correct(os.path.abspath(os.path.join(output_path, 'weights')))[:-1]) 
f.close()

# ===============================================train test
jpg = [i for i in os.listdir(data_dir) if i.endswith('jpg')]
txt = [i for i in os.listdir(data_dir) if i.endswith('txt') and i != "classes.txt"]
shuffle_id = list(range(len(jpg)))
random.shuffle(shuffle_id)
train = shuffle_id[: round(split_rate[0]*len(jpg))]
test = shuffle_id[round(split_rate[0]*len(jpg)) :]

for i in train:
    shutil.copy(os.path.join(data_dir, jpg[i]), os.path.join(output_path, 'train', jpg[i]))
    shutil.copy(os.path.join(data_dir, txt[i]), os.path.join(output_path, 'train', txt[i]))
for i in test:
    shutil.copy(os.path.join(data_dir, jpg[i]), os.path.join(output_path, 'test', jpg[i]))
    shutil.copy(os.path.join(data_dir, txt[i]), os.path.join(output_path, 'test', txt[i]))
    
# ===============================================train.txt test.txt
with open(os.path.join(output_path, 'cfg', cfg_train_txt), 'w') as f:
    for i in os.listdir(os.path.join(output_path, 'train')):
        if i.endswith('jpg'):
            f.write(path_correct(os.path.abspath(os.path.join(output_path, 'train', i)))[:-1] + "\n")
f.close()

with open(os.path.join(output_path, 'cfg', cfg_test_txt), 'w') as f:
    for i in os.listdir(os.path.join(output_path, 'test')):
        if i.endswith('jpg'):
            f.write(path_correct(os.path.abspath(os.path.join(output_path, 'test', i)))[:-1] + "\n")
f.close()